In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count
from pyspark.sql import functions as F

In [2]:
data_file_path = "./data/agg_match_stats_100000.csv"

# Create a SparkSession
spark = SparkSession.builder.appName("CSV Analysis").getOrCreate()

# Load the CSV file into a DataFrame
df = spark.read.csv(data_file_path, header=True, inferSchema=True)

all_columns = df.columns


In [3]:
# Show the schema of the DataFrame
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [4]:
# Show the first few rows of the DataFrame
df.show()

+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|               date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|   player_name|player_survive_time|team_id|team_placement|
+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|2017-11-26 21:59:40|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|      SnuffIes|            1106.32|      4|            18|
|2017-11-26 21:59:40|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|2938.407230

In [5]:
# Perform some basic operations
print("Number of rows: ", df.count())
print("Number of columns: ", len(df.columns))

# You can also perform more specific data analysis depending on your needs
# For example, to find the average of a column named 'column_name', you can do:
# df.select(avg("column_name")).show()

Number of rows:  99999
Number of columns:  15


# 3 - Les meilleurs joueurs

L'objectif est d'être le dernier en vie, mais certains joueurs soutiennent qu'il est nécessaire d'éliminer
un maximum de concurrents. Nous allons vérifier cette affirmation en comparant les joueurs selon ces deux
conditions, à vous de choisir celle que vous souhaitez explorer. L'attribut player_kills donne le nombre
d'éliminations et team_placement la position en fin de partie.

1. Chargez le jeu de données. (voir textFile)
2. Pour chaque partie, obtenez uniquement le nom du joueur et son nombre d'éliminations ou sa position. (voir map)
3. Obtenez la moyenne des éliminations ou de la position de chaque joueur, ainsi que le nombre de parties concernées. (voir reduceByKey ou groupByKey)
4. Obtenez les 10 meilleurs joueurs selon les éliminations ou la position. (voir sortBy)
5. Certains joueurs n'ayant joué qu'une partie, nous souhaitions ne garder que ceux ayant au moins 4 parties. (voir filter)
6. Si vous observez un joueur particulier, traitez-le de la manière appropriée.
7. En partageant avec vos camarades qui ont exploré l'autre condition, donnez votre avis sur l'affirmation de départ.

In [6]:
selected_data = df.select("player_name", "player_kills", "team_placement")
selected_data.show()

+--------------+------------+--------------+
|   player_name|player_kills|team_placement|
+--------------+------------+--------------+
|      SnuffIes|           1|            18|
|        Ozon3r|           1|            18|
|        bovize|           0|            33|
|       sbahn87|           0|            33|
|     GeminiZZZ|           2|            11|
|      AlexWho6|           0|            11|
|    R1dd1ck913|           0|            17|
|     Emmylou89|           0|            17|
|         ron-n|           0|            24|
|       SoSpicy|           1|            24|
|   MatthiasXIV|           1|             4|
|      Ninjapex|           0|             4|
|    thebrewski|           0|            34|
|GucciManeDrain|           2|            34|
|      subzero3|           1|             2|
|       Chogo82|           1|             2|
|    FIYMNBVCXZ|           0|             3|
|     ShuLanHou|           6|             3|
|         sigbo|           1|            20|
|       Je

In [7]:

player_stats = selected_data.groupBy("player_name").agg(
    avg("player_kills").alias("avg_kills"),
    avg("team_placement").alias("avg_placement"),
    count("player_name").alias("match_count")
)

player_stats.show()

+----------------+---------+------------------+-----------+
|     player_name|avg_kills|     avg_placement|match_count|
+----------------+---------+------------------+-----------+
|        Abby1998|      0.0|              29.0|          1|
|  mrflamingllama|      0.0|              23.0|          1|
|      Kasei_Chan|      0.5|              18.5|          2|
|   KenKen_Mantau|      0.0|              16.0|          1|
|        57Street|      1.0|              20.0|          1|
|      Joobjoobee|      2.0|              29.0|          1|
|   smegma_patrol|      0.0|              61.0|          1|
| speed_demonzzzz|      1.0|49.333333333333336|          3|
|    HuYa-Jun-Hao|      2.0|               9.0|          1|
|   ThangMatLon66|      0.0|              21.0|          1|
|snouted_snorkler|      0.0|              19.0|          1|
|          zhuoda|      0.0|              28.0|          1|
|    EricLawrence|      1.5|              12.0|          2|
|         I_Suck_|      5.0|            

In [8]:


top_players_by_kills = player_stats.sort("avg_kills", ascending=False).limit(20)
top_players_by_placement = player_stats.sort("avg_placement").limit(20)

top_players_by_kills.show()
top_players_by_placement.show()


+--------------+---------+-------------+-----------+
|   player_name|avg_kills|avg_placement|match_count|
+--------------+---------+-------------+-----------+
|      gogolnyg|     62.0|          1.0|          1|
|     651651646|     42.0|          1.0|          1|
|  appar1008611|     38.0|          1.0|          1|
|    EsNmToging|     36.0|          1.0|          1|
|       Kinmmpp|     30.0|          1.0|          1|
|      MoGu1314|     25.0|          1.0|          1|
| asdkmiojfdioe|     25.0|          1.0|          1|
|      motoMepp|     25.0|          1.0|          1|
|      KouBxczG|     24.0|          1.0|          1|
|   s1000r-race|     24.0|          4.0|          1|
|   LiliTusfdfs|     23.0|          1.0|          1|
|  babyylaowang|     23.0|          1.0|          1|
|  georgeking12|     22.0|          1.0|          1|
|     abbbbccc2|     22.0|          2.0|          1|
|QUN-25-9999250|     22.0|          1.0|          1|
| SR-HaoZheDaDa|     21.0|          1.0|      

In [9]:

filtered_stats = player_stats.filter("match_count >= 4")
filtered_stats.show()

+---------------+---------+-------------+-----------+
|    player_name|avg_kills|avg_placement|match_count|
+---------------+---------+-------------+-----------+
|         LILI-F|      0.0|         44.5|          4|
|   Informaldrip|     0.25|         23.5|          4|
|           immj|      0.4|         28.2|          5|
|        Dcc-ccD|     1.75|         14.5|          4|
|   Koreyoshi364|     0.25|        44.25|          4|
|    Roobydooble|      1.0|        27.75|          4|
|       dman4771|     1.75|         11.5|          4|
|      crazyone8|     0.25|        23.25|          4|
|        GenOrgg|      0.5|        48.75|          4|
|         KBSDUI|      0.0|         12.0|          4|
|   siliymaui125|      2.0|        22.75|          4|
|      TemcoEwok|     0.25|        13.25|          4|
|     PapaNuntis|      1.0|        13.25|          4|
|      NerdyMoJo|      1.5|        27.75|          4|
|  JustTuatuatua|     0.75|        10.75|          4|
| LawngD-a-w-n-g|      2.2| 

In [10]:

filtered_stats = df.filter("player_name = '651651646'")
filtered_stats.show()

+-------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+------------------+----------+------------+-----------+-------------------+-------+--------------+
|               date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|player_dist_ride|  player_dist_walk|player_dmg|player_kills|player_name|player_survive_time|team_id|team_placement|
+-------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+------------------+----------+------------+-----------+-------------------+-------+--------------+
|2018-01-07 13:59:03|       27|2U4GBNA0Yml3HEnAb...|       tpp|         4|             0|         32|             0.0|3287.7263199999998|      3956|          42|  651651646|           1286.271|      1|             1|
+-------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+--------------

In [11]:
print("number of top 1 in the dataset")
top_1 = player_stats.filter(player_stats.avg_placement == 1).count()
print(top_1)

print("Average kills for top 1")
avg_kills_top_1 = player_stats.filter(player_stats.avg_placement == 1).agg(avg("avg_kills"))
avg_kills_top_1.show()

number of top 1 in the dataset
2679
Average kills for top 1
+-----------------+
|   avg(avg_kills)|
+-----------------+
|3.710526315789474|
+-----------------+



The average winner has 4.1 kills which is quite low compared to the top killers of the matchs.
This means that doing kills does not necessarily mean winning the game.

Une observation intéressante est que les joueurs avec un grand nombre d'éliminations ont également tendance à bien se placer dans les matchs.

Meme si la plus part des gens dans le haut du classement ont un nombre d'elimination assez faible.

# 4 Score des joueurs

Nous allons assigner un score à chaque joueur selon ses actions lors de chaque partie. Chaque joueur
gagnerait 50 points par assistance, 1 point par dommage causé, 100 points par élimination et 1000 points
s'il finis à la première place, 990 à la deuxième, ainsi de suite.

1. Développez une fonction spécifique pour obtenir le score, et obtenez les 10 meilleurs joueurs selon ce critère.
2. Comparez ce classement avec les deux précédents critères

In [12]:
df = df.withColumn('score', 
                   50 * df['player_assists'] + 
                   df['player_dmg'] + 
                   100 * df['player_kills'] + 
                   (1000 - F.when(df['team_placement'] > 10, 0).otherwise(10 * (df['team_placement'] - 1))))

In [13]:
top_players = df.groupBy('player_name')\
                .sum('score')\
                .orderBy('sum(score)', ascending=False)\
                .limit(20)
# Remove null values
top_players = top_players.filter(top_players.player_name != 'NULL')
top_players.show()

+----------------+----------+
|     player_name|sum(score)|
+----------------+----------+
|            NULL|    169908|
|        gogolnyg|     13282|
|       651651646|      9156|
|    appar1008611|      8819|
|      EsNmToging|      8016|
|  LawngD-a-w-n-g|      7323|
|Hidden-In-Bushes|      6751|
|         Kinmmpp|      6629|
|        motoMepp|      6405|
|     LiliTusfdfs|      6332|
|   asdkmiojfdioe|      6028|
|    babyylaowang|      6009|
|        MoGu1314|      5941|
|    ILovefantasy|      5890|
|    georgeking12|      5846|
|        ViaMapie|      5756|
|        KouBxczG|      5743|
|         Dcc-ccD|      5709|
|            immj|      5579|
|    siliymaui125|      5511|
+----------------+----------+

